In [6]:
# se importa el conector de snowflake
import snowflake.connector 


In [7]:
#se crea conexión
conn = snowflake.connector.connect(
    user='grupods03',
    password='Henry2022#',
    account='nr28668.sa-east-1.aws')

In [8]:
def execute_query(connection, query):
    cursor = connection.cursor() # se inicializa la conexión Creates a cursor object. Each statement will be executed in a new cursor object.
    cursor.execute(query)
    cursor.close()

In [9]:
query = "use database PRUEBA" # se inicializa en database prueba

execute_query(conn, query)

In [ ]:
sql = 'create stage data_stage file_format = (type = "csv" field_delimiter = "," skip_header = 1)'
execute_query(conn, sql)

In [ ]:
csv_file = '/Users/josetoledo/Desktop/prueba/df_hecho.csv'
sql = "PUT file://" + csv_file + " @DATA_STAGE auto_compress=true"
    
execute_query(conn, sql)

In [1]:
import pandas as pd
import tempfile
import 
  


In [10]:
df = pd.DataFrame([(1,'America'), (2,'Europa'), (3,'Asia'),(4,'Africa'),(5,'Oceania')], columns=['ID_CONTINENTE', 'CONTINENTE']) # se crea dataframe con pandas


In [89]:
df

,ID_CONTINENTE,CONTINENTE
0,1,America
1,2,Europa
2,3,Asia
3,4,Africa
4,5,Oceania


In [99]:
file=df.to_csv(index=False)

In [37]:
with tempfile.TemporaryDirectory() as temp_dir:
    df.to_csv(temp_dir +'/continente3.csv', index=False)
    sql = f"PUT file://{temp_dir+'/continente3.csv'} @DATA_STAGE auto_compress=true"
    execute_query(conn, sql)


In [40]:
sql='list @DATA_STAGE'
cur=conn.cursor()
cur.execute(sql)
for c in cur:
    print(c)
cur.close()

('data_stage/continente.csv.gz', 96, '1e953ff09ae314adf434d4c6267fcd40', 'Tue, 18 Oct 2022 13:45:58 GMT')
('data_stage/continente2.csv.gz', 96, 'e02beef7c4a7d50b586656e6cae16cbb', 'Tue, 18 Oct 2022 13:46:30 GMT')
('data_stage/df_hecho.csv.gz', 127424, 'cd7e24237c3bd0a69976527fd4b7219c', 'Thu, 13 Oct 2022 18:20:38 GMT')
('data_stage/tmpo1gv29nlcontinente.csv.gz', 112, 'dd43809466184fbbedba4c43905058d8', 'Tue, 18 Oct 2022 13:44:58 GMT')
('data_stage/tmpxq_5c9d6continente.csv.gz', 112, '7329701b007f46b2e4ffe985d9fdd45e', 'Tue, 18 Oct 2022 13:45:09 GMT')
('data_stage/tmpz3m6_iqx.csv.gz', 96, '88121536ce05b77ed36c5a69d20434e7', 'Tue, 18 Oct 2022 13:08:44 GMT')


True

In [7]:
import logging
from datetime import datetime, timedelta
import airflow
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.providers.snowflake.hooks.snowflake import SnowflakeHook
from airflow.providers.snowflake.operators.snowflake import SnowflakeOperator

In [ ]:


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

args = {"owner": "Airflow", "start_date": datetime(2021,3,22,17,15)}

dag = DAG(
    dag_id="snowflake_connector3", default_args=args, schedule_interval=None
)

query1 = [
    """select 1;""",
    """show tables in database abcd_db;""",
]

def count1(**context):
    dwh_hook = SnowflakeHook(snowflake_conn_id="snowflake_conn")
    result = dwh_hook.get_first("select count(*) from abcd_db.public.test3")
    logging.info("Number of rows in `abcd_db.public.test3`  - %s", result[0])


with dag:
    query1_exec = SnowflakeOperator(
        task_id="snowfalke_task1",
        sql=query1,
        snowflake_conn_id="snowflake_conn",
    )

    count_query = PythonOperator(task_id="count_query", python_callable=count1)
query1_exec >> count_query


In [ ]:
#
# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied.  See the License for the
# specific language governing permissions and limitations
# under the License.
"""
Example use of Snowflake related operators.
"""
from __future__ import annotations

import os
from datetime import datetime

from airflow import DAG
from airflow.providers.snowflake.operators.snowflake import SnowflakeOperator


In [ ]:
#se crea conexión
conn = snowflake.connector.connect(
    user='grupods03',
    password='Henry2022#',
    account='nr28668.sa-east-1.aws')

In [ ]:


SNOWFLAKE_CONN_ID = 'my_snowflake_conn'
SNOWFLAKE_STAGE = 'stage_name'
SNOWFLAKE_WAREHOUSE = 'warehouse_name'
SNOWFLAKE_DATABASE = 'database_name'
SNOWFLAKE_ROLE = 'role_name'
SNOWFLAKE_SAMPLE_TABLE = 'sample_table'


GITHUB_FILE_PATH = '</datasets/sample_file.csv'

# SQL commands

ENV_ID = os.environ.get("SYSTEM_TESTS_ENV_ID")

DAG_ID = "example_snowflake"

# [START howto_operator_snowflake]

with DAG(
    DAG_ID,
    start_date=datetime(2021, 1, 1),
    default_args={'snowflake_conn_id': SNOWFLAKE_CONN_ID},
    tags=['example'],
    catchup=False,
) as dag:
    # [START snowflake_example_dag]

    snowflake_op_with_params = SnowflakeOperator(
        task_id='snowflake_op_with_params',
        sql=SQL_INSERT_STATEMENT,
        parameters={"id": 56},
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA,
        role=SNOWFLAKE_ROLE,
    )

    snowflake_op_sql_list = SnowflakeOperator(task_id='snowflake_op_sql_list', sql=SQL_LIST)

    snowflake_op_sql_multiple_stmts = SnowflakeOperator(
        task_id='snowflake_op_sql_multiple_stmts',
        sql=SQL_MULTIPLE_STMTS,
    )

    snowflake_op_template_file = SnowflakeOperator(
        task_id='snowflake_op_template_file',
        sql='/path/to/sql/<filename>.sql',
    )

    # [END howto_operator_snowflake]

    (
        snowflake_op_sql_str
        >> [
            snowflake_op_with_params,
            snowflake_op_sql_list,
            snowflake_op_template_file,
        
            snowflake_op_sql_multiple_stmts,
        ]
    
    )
    # [END snowflake_example_dag]





In [ ]:



SNOWFLAKE_CONN_ID = 'my_snowflake_conn'

# TODO: should be able to rely on connection's schema, but currently param required by S3ToSnowflakeTransfer

SNOWFLAKE_SCHEMA = 'schema_name'
SNOWFLAKE_STAGE = 'stage_name'
SNOWFLAKE_WAREHOUSE = 'warehouse_name'
SNOWFLAKE_DATABASE = 'database_name'
SNOWFLAKE_ROLE = 'role_name'
SNOWFLAKE_SAMPLE_TABLE = 'sample_table'


GITHUB_FILE_PATH = '</datasets/sample_file.csv'

# SQL commands

CREATE_TABLE_SQL_STRING = (
    f"CREATE OR REPLACE TRANSIENT TABLE {SNOWFLAKE_SAMPLE_TABLE} (name VARCHAR(250), id INT);"

)

SQL_INSERT_STATEMENT = f"INSERT INTO {SNOWFLAKE_SAMPLE_TABLE} VALUES ('name', %(id)s)"

SQL_LIST = [SQL_INSERT_STATEMENT % {"id": n} for n in range(0, 10)]

SQL_MULTIPLE_STMTS = "; ".join(SQL_LIST)


SNOWFLAKE_SLACK_SQL = f"SELECT name, id FROM {SNOWFLAKE_SAMPLE_TABLE} LIMIT 10;"


SNOWFLAKE_SLACK_MESSAGE = (
    "Results in an ASCII table:\n```{{ results_df | tabulate(tablefmt='pretty', headers='keys') }}```"

)

ENV_ID = os.environ.get("SYSTEM_TESTS_ENV_ID")


DAG_ID = "example_snowflake"


# [START howto_operator_snowflake]

with DAG(
    DAG_ID,
    start_date=datetime(2021, 1, 1),
    default_args={'snowflake_conn_id': SNOWFLAKE_CONN_ID},
    tags=['example'],
    catchup=False,
) as dag:
    # [START snowflake_example_dag]

    snowflake_op_sql_str = SnowflakeOperator(
        task_id='snowflake_op_sql_str',
        sql=CREATE_TABLE_SQL_STRING,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA,
        role=SNOWFLAKE_ROLE,

    )

    snowflake_op_with_params = SnowflakeOperator(
        task_id='snowflake_op_with_params',
        sql=SQL_INSERT_STATEMENT,
        parameters={"id": 56},
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA,
        role=SNOWFLAKE_ROLE,
    )

    snowflake_op_sql_list = SnowflakeOperator(task_id='snowflake_op_sql_list', sql=SQL_LIST)

    snowflake_op_sql_multiple_stmts = SnowflakeOperator(
        task_id='snowflake_op_sql_multiple_stmts',
        sql=SQL_MULTIPLE_STMTS,
    )

    snowflake_op_template_file = SnowflakeOperator(
        task_id='snowflake_op_template_file',
        sql='/path/to/sql/<filename>.sql',
    )

    # [END howto_operator_snowflake]

    (
        snowflake_op_sql_str
        >> [
            snowflake_op_with_params,
            snowflake_op_sql_list,
            snowflake_op_template_file,
        
            snowflake_op_sql_multiple_stmts,
        ]
    
    )
    # [END snowflake_example_dag]





In [ ]:
from tests.system.utils import get_test_run  # noqa: E402

# Needed to run the example DAG with pytest (see: tests/system/README.md#run_via_pytest)

test_run = get_test_run(dag)

In [ ]:
import logging
from datetime import datetime, timedelta
import airflow
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.providers.snowflake.hooks.snowflake import SnowflakeHook
from airflow.providers.snowflake.operators.snowflake import SnowflakeOperator

In [ ]:
import requests
import json
import snowflake.connector
from datetime import datetime, timedelta
from airflow import DAG
from airflow.hooks.base_hook import BaseHook
from airflow.operators.python_operator import PythonOperator


In [ ]:
# credentials
snowflake_username = BaseHook.get_connection('snowflake').login
snowflake_password = BaseHook.get_connection('snowflake').password
snowflake_account = BaseHook.get_connection('snowflake').host



In [42]:
from airflow import DAG
from airflow.operators.python import PythonOperator, BranchPythonOperator
from airflow.operators.bash import BashOperator
from datetime import datetime

In [44]:
def create_wb_hecho_df():
    df = pd.DataFrame([(1,'America'), (2,'Europa'), (3,'Asia'),(4,'Africa'),(5,'Oceania')], columns=['ID_CONTINENTE', 'CONTINENTE']) # se crea dataframe con pandas
    return df

def upload_to_snowflake():
    with tempfile.TemporaryDirectory() as temp_dir:
        df.to_csv(temp_dir +'/wb_hecho.csv', index=False)
        sql = f"PUT file://{temp_dir+'/wb_hecho.csv'} @DATA_STAGE auto_compress=true"
        execute_query(conn, 'USE DATABASE PRUEBA')
        execute_query(conn, sql)

with DAG("my_dag", # Dag id
    start_date=datetime(2022, 10 ,19), # start date, the 1st of January 2021 
    schedule='@yearly',  # Cron expression, here it is a preset of Airflow, @yearly means once every year.
    catchup=False  # Catchup 
    ) as dag:

        create_wb_hecho=PythonOperator(
            task_id="creation_of_dataframe_wb",
            python_callable=create_wb_hecho_df)
            
        put_file_into_stage_snowflake=PythonOperator(
            task_id="file_to_snowflake_stage",
            python_callable=upload_to_snowflake)

        create_wb_hecho >> put_file_into_stage_snowflake

In [ ]:
import snowflake.connector

conn = snowflake.connector.connect(
    user='grupods03',
    password='Henry2022#',
    account='nr28668.sa-east-1.aws')

In [ ]:


def execute_query(connection, query):
    cursor = connection.cursor() # se inicializa la conexión Creates a cursor object. Each statement will be executed in a new cursor object.
    cursor.execute(query)
    cursor.close()

    
dir='https://raw.githubusercontent.com/grupohenryds03/esperanza_vida/main/datasets/hechos.csv?token=GHSAT0AAAAAAB2ENG2ZQEV5BRDAG5BZT6TIY2P7M7Q'
sql = f"PUT file://{dir} @DATA_STAGE auto_compress=true"
execute_query(conn, 'USE DATABASE PRUEBA')
execute_query(conn, sql)